In [1]:
import torch
import torch.nn.functional as F
from layer import TransformerBlock, HierarchicalTransformerBlock

%load_ext autoreload
%autoreload 2

In [ ]:
768 * 4 * 2/3
# d_ff = d_model * 4 * 2/3

2048.0

In [ ]:
768 / 8
# d_k = d_model / h

96.0

#### test transformer block

In [2]:
encoder = TransformerBlock(
    d_model=512,
    d_ff=1280,
    h=8,
    dropout=0,
    norm_type="layer",
    ffn_type="swiglu",
)

In [3]:
encoder = encoder.to("cuda")
encoder

TransformerBlock(
  (self_attn_block): MultiHeadAttentionBlock(
    (w_q): Linear(in_features=512, out_features=512, bias=False)
    (w_k): Linear(in_features=512, out_features=512, bias=False)
    (w_v): Linear(in_features=512, out_features=512, bias=False)
    (w_o): Linear(in_features=512, out_features=512, bias=False)
  )
  (ffn_block): FFNSwiGLUBlock(
    (linear_gate): Linear(in_features=512, out_features=1280, bias=True)
    (linear_up): Linear(in_features=512, out_features=1280, bias=True)
    (linear_down): Linear(in_features=1280, out_features=512, bias=True)
  )
  (residual_connection): ResidualConnection(
    (dropout): Dropout(p=0, inplace=False)
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)

In [4]:
inputs = torch.randn(2, 128, 512).to("cuda")
masks = torch.ones(2, 128).to("cuda")

In [5]:
encoder(inputs, masks).shape

torch.Size([2, 128, 512])

#### test hierarchical transformer block

In [2]:
hat_block = HierarchicalTransformerBlock(
    d_model=768,
    d_ff=2048,
    h=8,
    dropout=0,
    norm_type="layer",
    ffn_type="gelu",
).cuda()

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(hat_block) / 1e6

11.018752

In [4]:
(2e4 * 768 + 11e6 * 12) / 1e6

147.36

In [3]:
seg_hidden_state = torch.randn(64, 32, 128, 768).cuda()
token_mask = torch.rand(64, 32, 128).bernoulli().cuda()
seg_mask = torch.rand(64, 32).bernoulli().cuda()


In [4]:
hat_block(seg_hidden_state, token_mask, seg_mask).shape

torch.Size([64, 32, 128, 768])

#### collate batch

In [ ]:
tokens = [
    ['cls', 'a', 'b', 'c'],
    ['cls', 'd', 'e'],
    ['cls', 'f', 'g', 'h', 'i'],
]